In [1]:
import time
from urllib.request import urlopen
from urllib.request import Request
import json
import requests
import os
import re

In [2]:
# 请求头
headers = {
    'Cookie': '_octo=GH1.1.547050398.1730121222; _device_id=73a715dea9188e9052280e9ba8f00a3d; saved_user_sessions=124904356%3AFGDwEe-yLWPAB9iCFsxyTgm-Ij8frauGVZNDils0DyQ_FDi4; user_session=FGDwEe-yLWPAB9iCFsxyTgm-Ij8frauGVZNDils0DyQ_FDi4; __Host-user_session_same_site=FGDwEe-yLWPAB9iCFsxyTgm-Ij8frauGVZNDils0DyQ_FDi4; logged_in=yes; dotcom_user=yizhilsy; color_mode=%7B%22color_mode%22%3A%22auto%22%2C%22light_theme%22%3A%7B%22name%22%3A%22light%22%2C%22color_mode%22%3A%22light%22%7D%2C%22dark_theme%22%3A%7B%22name%22%3A%22dark%22%2C%22color_mode%22%3A%22dark%22%7D%7D; preferred_color_mode=light; tz=Asia%2FShanghai; _gh_sess=wgcGGu1teQcioqHPiWBCx0tVV%2BgI4OSl5S1WSmwI4%2BLlrg7DS2GFaw1Z%2BTJRYq4SKGHp%2Fm4b9ZkIfhQqgACyapyAa8XZtdkOIGF%2Fkk%2BTwIj9ibdWec5%2B8oofaPsQipgRe7Jmfv3LPdqazTY%2BxAj8IujwSpfd%2BSaJqI3QPiZucUbs%2FYPRwWQfcz7qWrb0%2Ftzbd0eeLfAQg75s1uxusz67iprHm%2Fl0rFU6ZOrzMB1Gl4uVPDXP%2FkOwpiCLFvgWUaKEgTAEy2C4T%2BMTFL285JZ1cDY1rM0TPhOkU89zIezfnw6YflOnC%2FE%2FhgRne3QOWKI1mAvd%2BleJNC3EE0nVhVZwC5I%2FQw0%2BuJGikotXbvSc0MWRrHsaD1Nen3gR6WAchiW6--2m9XRat%2B2oaDceNY--FmGSxVluLBuvzl1Y7CQe5A%3D%3D',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Accept': 'text/html',
    'Sec-Ch-Ua': '"Not?A_Brand";v="99", "Chromium";v="130"',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.59 Safari/537.36',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://github.com/topics',
    'Accept-Encoding': 'gzip, deflate, br'
}
def get_Topics(page):
    # URL
    url = 'https://github.com/topics?page={page}'.format(page = page)
    response = requests.get(url, headers=headers)
    # 检查请求是否成功
    if response.status_code == 200:
        return response.text    # 返回 HTML 内容
    else:
        print(f"请求失败，状态码：{response.status_code}")
        return None

In [3]:
# 获取所有官方topic的html文件（6页page）
for page in range(1, 7):
    topic_html = get_Topics(page)
    with open("./topicPages/topics_page_{}.html".format(page), "w", encoding="utf-8") as f:
        f.write(topic_html)

In [5]:
# 读取爬取的html文件分割出官方topic字符串存入文件中
# 设置文件目录
directory = "./topicPages/"
# 正则表达式模式
pattern = r'<p class="f3 lh-condensed mb-0 mt-1 Link--primary">(.*?)</p>'
topics_list = []
for page in range(1, 7):
    filename = f"topics_page_{page}.html"
    filepath = os.path.join(directory, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()
        # 查找所有匹配的字符串
        matches = re.findall(pattern, content)
        if matches:
            for match in matches:
                topics_list.append(match)
# 写入文件
with open("./topicPages/govTopics.txt", "a", encoding="utf-8") as f:
    count = 1
    for topic in topics_list:
        f.write(str(count) + ":" + topic + "\n")
        count = count + 1
print(count-1)

166
